In [ ]:
%load_ext autoreload
%autoreload 2

%cd '..'

In [ ]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("data/logs/events_pen_polarization.log"),
        logging.StreamHandler(stream=sys.stdout)
    ]
)

In [ ]:
import seaborn as sns

sns.set_theme()
from load.utils import load_users
from preprocessing.utils import (
    load_event_comments,
    load_event_vocab,
)
from preprocessing.constants import (
    ELECTIONS_EVENTS_INFO,
)


In [ ]:
EVENT_KEY = "us_elections_2016"
EVENT_DETAILS = ELECTIONS_EVENTS_INFO[EVENT_KEY]

logging.info(EVENT_DETAILS)

In [ ]:
users = load_users(engine="polars")

In [ ]:
users

In [ ]:
event_comments = load_event_comments(event_name=EVENT_KEY, engine="pandas")
event_vocab = load_event_vocab(event_name=EVENT_KEY)

In [ ]:
event_authors = event_comments[["author", "party"]].drop_duplicates(subset=["author"])


In [ ]:
event_author_features = users.merge(
    event_authors["author"],
    how="inner",
    left_on="author",
    right_on="author",
)

event_author_features = event_author_features.set_index("author")

In [ ]:
event_author_features

In [ ]:
event_author_features["gender"] = event_author_features["gender"].astype("str").replace(
    {"": 0, "f": 1, "m": 2},
).astype("int64")


In [ ]:
X = event_author_features.to_numpy(dtype=int)

In [ ]:
author_party = users[["author"]].merge(
    event_authors,
    how="inner",
    left_on="author",
    right_on="author",
)

In [ ]:
is_republican = (author_party["party"] == "rep").to_numpy(dtype=int)